# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 13 2023 9:36AM,257216,12,9152810,"SD Head USA, LLC",Released
1,Feb 13 2023 9:22AM,257204,10,MSP218422,"Methapharm, Inc.",Released
2,Feb 13 2023 9:22AM,257204,10,MSP218423,"Methapharm, Inc.",Released
3,Feb 13 2023 9:22AM,257204,10,MSP218424,"Methapharm, Inc.",Released
4,Feb 13 2023 9:22AM,257204,10,MSP218425,"Methapharm, Inc.",Released
5,Feb 13 2023 9:22AM,257204,10,MSP218427,"Methapharm, Inc.",Released
6,Feb 13 2023 9:22AM,257204,10,MSP218428,"Methapharm, Inc.",Released
7,Feb 13 2023 9:22AM,257204,10,MSP218429,"Methapharm, Inc.",Released
8,Feb 13 2023 9:22AM,257204,10,MSP218435,"Methapharm, Inc.",Released
9,Feb 13 2023 9:22AM,257204,10,MSP218437,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,257210,Released,45
22,257211,Released,19
23,257212,Released,1
24,257213,Released,2
25,257216,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257210,NaN,45.0
257211,NaN,19.0
257212,NaN,1.0
257213,NaN,2.0
257216,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257179,0.0,1.0
257186,0.0,3.0
257191,0.0,12.0
257193,25.0,1.0
257194,34.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257179,0,1
257186,0,3
257191,0,12
257193,25,1
257194,34,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257179,0,1
1,257186,0,3
2,257191,0,12
3,257193,25,1
4,257194,34,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257179,,1
1,257186,,3
2,257191,,12
3,257193,25,1
4,257194,34,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 13 2023 9:36AM,257216,12,"SD Head USA, LLC"
1,Feb 13 2023 9:22AM,257204,10,"Methapharm, Inc."
36,Feb 13 2023 9:22AM,257210,10,ISDIN Corporation
81,Feb 13 2023 9:22AM,257209,10,ISDIN Corporation
128,Feb 13 2023 9:21AM,257213,10,"CLINUVEL, Inc."
130,Feb 13 2023 9:20AM,257208,10,ISDIN Corporation
163,Feb 13 2023 9:20AM,257212,10,ISDIN Corporation
164,Feb 13 2023 9:20AM,257211,10,"Digital Brands, LLC"
183,Feb 13 2023 9:19AM,257206,10,ISDIN Corporation
244,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 13 2023 9:36AM,257216,12,"SD Head USA, LLC",,1
1,Feb 13 2023 9:22AM,257204,10,"Methapharm, Inc.",,35
2,Feb 13 2023 9:22AM,257210,10,ISDIN Corporation,,45
3,Feb 13 2023 9:22AM,257209,10,ISDIN Corporation,,47
4,Feb 13 2023 9:21AM,257213,10,"CLINUVEL, Inc.",,2
5,Feb 13 2023 9:20AM,257208,10,ISDIN Corporation,,33
6,Feb 13 2023 9:20AM,257212,10,ISDIN Corporation,,1
7,Feb 13 2023 9:20AM,257211,10,"Digital Brands, LLC",,19
8,Feb 13 2023 9:19AM,257206,10,ISDIN Corporation,,61
9,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 9:36AM,257216,12,"SD Head USA, LLC",1,
1,Feb 13 2023 9:22AM,257204,10,"Methapharm, Inc.",35,
2,Feb 13 2023 9:22AM,257210,10,ISDIN Corporation,45,
3,Feb 13 2023 9:22AM,257209,10,ISDIN Corporation,47,
4,Feb 13 2023 9:21AM,257213,10,"CLINUVEL, Inc.",2,
5,Feb 13 2023 9:20AM,257208,10,ISDIN Corporation,33,
6,Feb 13 2023 9:20AM,257212,10,ISDIN Corporation,1,
7,Feb 13 2023 9:20AM,257211,10,"Digital Brands, LLC",19,
8,Feb 13 2023 9:19AM,257206,10,ISDIN Corporation,61,
9,Feb 13 2023 9:11AM,257207,12,"Uniderm USA, Inc",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 9:36AM,257216,12,"SD Head USA, LLC",1,
1,Feb 13 2023 9:22AM,257204,10,"Methapharm, Inc.",35,
2,Feb 13 2023 9:22AM,257210,10,ISDIN Corporation,45,
3,Feb 13 2023 9:22AM,257209,10,ISDIN Corporation,47,
4,Feb 13 2023 9:21AM,257213,10,"CLINUVEL, Inc.",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 9:36AM,257216,12,"SD Head USA, LLC",1,NaN
1,Feb 13 2023 9:22AM,257204,10,"Methapharm, Inc.",35,NaN
2,Feb 13 2023 9:22AM,257210,10,ISDIN Corporation,45,NaN
3,Feb 13 2023 9:22AM,257209,10,ISDIN Corporation,47,NaN
4,Feb 13 2023 9:21AM,257213,10,"CLINUVEL, Inc.",2,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 9:36AM,257216,12,"SD Head USA, LLC",1,0.0
1,Feb 13 2023 9:22AM,257204,10,"Methapharm, Inc.",35,0.0
2,Feb 13 2023 9:22AM,257210,10,ISDIN Corporation,45,0.0
3,Feb 13 2023 9:22AM,257209,10,ISDIN Corporation,47,0.0
4,Feb 13 2023 9:21AM,257213,10,"CLINUVEL, Inc.",2,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3858046,298,0.0
12,771619,10,43.0
19,1285971,63,59.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3858046,298,0.0
1,12,771619,10,43.0
2,19,1285971,63,59.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,298,0.0
1,12,10,43.0
2,19,63,59.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,298.0
1,12,Released,10.0
2,19,Released,63.0
3,10,Executing,0.0
4,12,Executing,43.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19
Status,,,
Executing,0.0,43.0,59.0
Released,298.0,10.0,63.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19
0,Executing,0.0,43.0,59.0
1,Released,298.0,10.0,63.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19
0,Executing,0.0,43.0,59.0
1,Released,298.0,10.0,63.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()